# SQL

## Introduction

- pronunciation: "S.Q.L" or "sequel";
- supported by all major commercial database systems;
- standardized: many new features over time;
- interactive via GUI or prompt, or embedded in programs;
- declarative, based on relational algebra;
- terminology and commands:
    - Data Definition Language: DDL;
    - Data Manipulation Language: DML;
    - Other commands:
        - indexes
        - constraints
        - views
        - triggers
        - transactions
        - authorization, ...

## Basic SELECT

- Basic Select:

```sql
SELECT A1, A2, ..., An
    FROM   R1, R2, ..., Rm  -- expression involl Table Variables
    WHERE  conditions       -- expression 
    ORDER BY attributes
```

- Demo Data:

```sql
/* Demo Data */
College(cName, state, enrollment)
Student(sID, sName, GPA, sizeHS)
Apply(sID, cName, major, decision)
```

- Demos:

```sql
/* Pi (sID, sName, GPA) sigma (GPA > 3.6) (Student) */
SELECT sID, sName, GPA
    FROM Student
    WHERE GPA > 3.6;

/* Pi (sName, major) sigma (Student.sID = Apply.sID) (Student * Apply) */ -- no duplicate in relational algebra
SELECT sName, major                 -- can get duplicated records
    FROM Student, Apply
    WHERE Student.sID = Apply.sID;  
SELECT distinct sName, major
    FROM Student, Apply
    WHERE Student.sID = Apply.sID;  -- no duplicate

/*
Pi (sName, GPA, decision)
    sigma (Student.sID = Apply.sID ^ sizeHS < 1000 ^ major = 'CS' ^ cName = 'Stanford')
        (Student * Apply)
*/
SELECT sName, GPA, decision
    FROM Student, Apply
    WHERE Student.sID = Apply.sID
        and sizeHS < 1000
        and major = 'CS'
        and cName = 'Stanford';

/*
Pi (cName)
    sigma (College.cName = Apply.cName ^ enrollment > 1000 ^ major = 'CS')
        (College * Apply)
*/
SELECT cName                        -- will return ERROR
    FROM College, Apply
    WHERE College.cName = Apply.cName
        and enrollment > 1000
        and major = 'CS';
SELECT distinct College.cName       -- the right way
    FROM College, Apply
    WHERE College.cName = Apply.cName
        and enrollment > 1000
        and major = 'CS';
        
/*
Pi (sID, sName, GPA, cName, enrollment)
    sigma (Apply.sID = Student.sID ^ Apply.cName = College.cName)
        (Student * (Apply * College))
*/
SELECT distinct Student.sID, sNAme, GPA, College.cName, enrollment
    FROM Student, Apply, College
    WHERE Apply.sID = Student.sID and Apply.cName = College.cName
        and College.major like '%bio%'   -- you can use wildcard in SQL
    ORDER BY GPA desc, enrollment;       -- you can sort in SQL
    

/*
Calculate attributes and rename
*/
SELECT sID, sName, GPA, sizeHS, GPA*(sizeHS/1000.0) as scaledGPA
FROM Student;
```

## Table Variables and Set Operators

```sql
/* Demo Data */
College(cName, state, enrollment)
Student(sID, sName, GPA, sizeHS)
Apply(sID, cName, major, decision)
```

### Rename

```sql
/*
sigma (A.sID = S.sID ^ A.cName = C.cName)
    (rho (A)(sID, cName)(Apply) * rho (S)(sID, sName, GPA)(Student) * rho (C)(cName, enrollment)(College))
*/
SELECT distinct S.sID, sNAme, GPA, C.cName, enrollment
    FROM Student S, Apply A, College C                 -- Rename
    WHERE A.sID = S.sID and A.cName = C.cName


/* Find students with the same GPA
sigma (S1.GPA = S2.GPA ^ S1.sID < S2.sID)
    (rho (S1)(sID, sName, GPA)(Student) * rho (S2)(sID, sName, GPA)(Student))
*/
SELECT distinct S1.sID, S1.sName, S1.GPA, S2.sID, S2.sName, S2.GPA
    FROM Student S1, Student S2
    WHERE S1.GPA = S2.GPA and S1.sID < S2.sID; -- sID is different
```

### Set Operation
    
    
```sql
/* List of college and student names
(rho c(name) (Pi (cName) (College))) union (rho c(name) (Pi (sName) (Student)))
*/
SELECT cName as name From College
union                             -- eliminate duplications by default in SQLite, append `all` to keep all.
SELECT sName as name From Student
    ORDER BY name
    
    
/* List sID of students select both `CS` and `EE`
*/
SELECT sID FROM Student WHERE major = 'CS'
intersect
SELECT sID FROM Student WHERE major = 'EE'

-- same as above:
SELECT distinct A1.sID
    FROM Apply A1, Apply A2
        WHERE A1.sID = A2.sID and A1.major = 'CS' and A2.major = 'EE'
       
       
/* List sID of students select `CS` but not `EE`
*/
SELECT sID FROM Apply WHERE major = 'CS'
except
SELECT sID FROM Apply WHERE major = 'EE'

-- the result may be different from above:
SELECT distinct A1.sID
    FROM Apply A1, Apply A2
        WHERE A1.sID = A2.sID and A1.major = 'CS' and A2.major <> 'EE'
```

## Subqueries in WHERE Clause

```sql
/* Demo Data */
College(cName, state, enrollment)
Student(sID, sName, GPA, sizeHS)
Apply(sID, cName, major, decision)
```



```sql
SELECT distinct sID, sName
FROM Student, Apply
WHERE Student.sID = Apply.sID and major = 'CS';


-- VS


SELECT sID, sName
FROM Student
WHERE sID in (SELECT sID FROM Apply WHERE major = 'CS');
```



```sql
SELECT distinct sName
FROM Student, Apply
WHERE Student.sID = Apply.sID and major = 'CS';


-- VS


SELECT sName
FROM Student
WHERE sID in (SELECT sID FROM Apply WHERE major = 'CS');
```


**duplicates is different**

- Duplicte matters, the only way to get right duplicates is using sub queries in WHERE clause!


### In, and Not

```sql
/* 
Find students applied for CS but not EE
*/
SELECT sID, sName
FROM Student
WHERE sID in (SELECT sID FROM Apply WHERE major = 'CS')
    and sID not in (SELECT sID FROM Apply WHERE major = 'EE');
```

### Exists, All, and Any

```sql
/* 
Find all colleges that has some other college in the same state
*/
SELECT cName, state
FROM College C1
WHERE exists (SELECT * FROM College C2 WHERE C2.state = C1.state and C2.cName <> C1.cName);
```

```sql
/* 
Find college with the largest enrollment
*/
SELECT cName
FROM College C1
WHERE not exists (SELECT * FROM College C2 WHERE C2.enrollment > C1.enrollment);

-- Same result with `all`, if only one highest exists

SELECT cName
FROM College C1
WHERE enrollment > all (SELECT enrollment FROM College C2 WHERE C1.cName <> C2.cName)


-- Same result with `any`

SELECT cName
FROM College C1
WHERE not enrollment <= any (SELECT enrollment FROM College C2 WHERE C1.cName <> C2.cName)
```

```sql
/* 
Find student with the highest GPA
*/
SELECT sName, GPA
FROM Student S1
WHERE not exists (SELECT * FROM Student S2 WHERE S2.GPA > S1.GPA)

-- Same As

SELECT sName, GPA
FROM Student S1
WHERE GPA >= all (SELECT GPA FROM Student)

-- VS

SELECT sName, GPA
FROM Student S1, Student S2
WHERE S1.GPA > S2.GPA
-- Find the students that have some other student has lower GPA then them.
```



```sql
/* 
Find students not from the smallest high school
*/
SELECT sID, sName, sizeHS
FROM Student
WHERE sizeHS > any (SELECT sizeHS FROM Student)

-- Same as

SELECT sID, sName, sizeHS
FROM Student S1
WHERE exists (SELECT * FROM Student S2 WHERE S2.sizeHS < S1.sizeHS)
-- We can use `exists` to represent any `any` or `all`
```



```sql
/* 
Find students who applied CS but not EE
*/
SELECT sID, sName
FROM Student
WHERE sID = any (SELECT sID FROM Apply WHERE major = 'CS')
    and sID <> any (SELECT sID FROM Apply WHERE major = 'EE');
```

### SQL: Subquries in FROM and SELECT

```sql
/* Demo Data */
College(cName, state, enrollment)
Student(sID, sName, GPA, sizeHS)
Apply(sID, cName, major, decision)
```

- Subquery in FROM

```sql
SELECT sID, sName, GPA, GPA*(sizeHS/1000.0) as scaledGPA
FROM Student
WHERE abs(GPA*(sizeHS/1000.0) - GPA) > 1

-- Same as:
SELECT * 
FROM (
    SELECT sID, sName, GPA, GPA*(sizeHS/1000.0) as scaledGPA
    FROM Student
) as G
WHERE abs(G.scaledGPA - GPA) > 1
```

- Subquery in SELECT

```sql
/*
Colleges paired with the highest GPA of their applicants
*/
SELECT cName, state, ( SELECT distinct GPA
    FROM APPly, Student
    WHERE College.cName = Apply.cName and Student.sID = Apply.sID
        and GPA >= all(SELECT GPA 
        FROM Student, Apply
        WHERE Student.sID = Apply.sID and College.cName = Apply.cName)) as GPA
FROM College
```

## The JOIN Family of Operators

- Some time can imporve performance using join operators.



```sql
/* Demo Data */
College(cName, state, enrollment)
Student(sID, sName, GPA, sizeHS)
Apply(sID, cName, major, decision)
```


### inner join on condition

- like algebra theta join in relational algebra

```sql
SELECT distinct sName, major
FROM Student, Apply
WHERE Student.sID = Apply.sID;

-- Same as:

SELECT distinct sName, major
FROM Student join Apply          -- inner join is the default join
    on Student.sID = Apply.sID 
```

```sql
/*
*/
SELECT sName, GPA
FROM Student, Apply
WHERE Student.sID = Apply.sID
    and sizeHS < 1000 and major = 'CS' and cName = 'Stanford';
    
-- Same as:

SELECT sName, GPA
FROM Student join Apply on Student.sID = Apply.sID
WHERE sizeHS < 1000 and major = 'CS' and cName = 'Stanford';

-- Same as:

SELECT sName, GPA
FROM Student join Apply on Student.sID = Apply.sID
    and sizeHS < 1000 and major = 'CS' and cName = 'Stanford';
```

```sql
/*
*/
SELECT Apply.sID, sName, GPA, Apply.cName, enrollment
FROM Apply, Student, College
WHERE Apply.sID = Student.sID and Apply.cName = College.cName;

-- Following will ERROR:

SELECT Apply.sID, sName, GPA, Apply.cName, enrollment
FROM Apply join Student join College             -- multi join is not allowed
     on Apply.sID = Student.sID and Apply.cName = College.cName;
     
-- Should be:

SELECT Apply.sID, sName, GPA, Apply.cName, enrollment
FROM (Apply join Student on Apply.sID = Student.sID) join College             -- multi join is not allowed
     on Apply.cName = College.cName;
```


### natural join

```sql
SELECT distinct Apply.sID, sName, major
FROM Student inner join Apply
    on Student.sID = Apply.sID;
    
-- Same as:

SELECT sID, sName, major           -- will eliminate duplicated rows and columns without ambiguity
FROM Student natural join Apply;   -- because sID is the only key in common


/*
*/
SELECT * FROM Student S1 natural join Student S2;

-- Same as:

SELECT * FROM Student;
```


### inner join using attrs



```sql
/*
*/
SELECT sName, GPA
FROM Student join Apply
on Student.sID = Apply.sID
WHERE sizeHS < 1000 and major = 'CS' and cName = 'Stanford';

-- Same as:

SELECT sName, GPA
FROM Student natural join Apply
WHERE sizeHS < 1000 and major = 'CS' and cName = 'Stanford';

-- Better:

SELECT sName, GPA
FROM Student join Apply using(sID)
WHERE sizeHS < 1000 and major = 'CS' and cName = 'Stanford';   -- `using` can only work with `WHERE`


/*
*/
SELECT S1.sID, S1.sName, S1.GPA, S2.sID, S2.sName, S2.GPA
FROM Student S1, Student S2
WHERE S1.GPA = S2.GPA and S1.sID < S2.sID;

-- Same as:

SELECT S1.sID, S1.sName, S1.GPA, S2.sID, S2.sName, S2.GPA
FROM Student S1 join Student S2 using(GPA)
WHERE S1.sID < S2.sID;
```



### left | right | full outer join


- left

```sql
/*
Students has applied some college
*/
SELECT sName, sID, cName, major
FROM Student inner join Apply using(sID)

-- VS:

/*
All Students no matter applied or not applied
*/
SELECT sName, sID, cName, major
FROM Student left join Apply using(sID)  -- `left join` is the abbr. for `left outer join`

-- Same as:

SELECT sName, sID, cName, major
FROM Student natural left outer join Apply     -- natural join on sID

-- Same as:

SELECT sName, Student.sID, cName, major
FROM Student, Apply
WHERE Student.sID = Apply.sID
union
SELECT sName, Student.sID, NULL, NULL
FROM Student
WHERE sID not in (SELECT sID FROM Apply);
```

- right

```sql
/*
All majors in Colleges no matter got or not got applicants
*/
SELECT sName, sID, cName, major
FROM Student right join Apply using(sID)
```

- full

```sql
/*
Cartesian product
*/
SELECT sName, sID, cName, major
FROM Student full join Apply using(sID);

-- Same as:

SELECT sName, sID, cName, major
FROM Student left join Apply using(sID)
union
SELECT sName, sID, cName, major
FROM Student right join Apply using(sID);

-- Same as:

SELECT sName, Student.sID, cName, major
FROM Student, Apply
WHERE Student.sID = Apply.sID
union
SELECT sName, Student.sID, NULL, NULL
FROM Student
WHERE sID not in (SELECT sID FROM Apply)
union
SELECT NULL, Student.sID, cName, major
FROM Apply
WHERE sID not in (SELECT sID FROM Student);
```

- pitfall of outer join:
    - like `Cartesian product`, order matters in multi outer join.


## Aggregation

```sql 
SELECT A1, A2, ..., An      -- min, max, count, sum, avg
    FROM   R1, R2, ..., Rm  -- expression involl Table Variables
    WHERE  conditions       -- expression 
    GROUP BY attributes
    HAVING conditions
```


```sql
/* Demo Data */
College(cName, state, enrollment)
Student(sID, sName, GPA, sizeHS)
Apply(sID, cName, major, decision)
```

- avg, min, max

```sql
/*
Average GPA of All students
*/
SELECT avg(GPA)
FROM Student

/*
Lowest GPA of students who applied CS
*/
SELECT min(GPA)
FROM Student, Apply
WHERE Student.sID = Apply.sID and major = 'CS'

/*
Average GPA of students who applied CS
*/
SELECT avg(GPA)
FROM Student, Apply
WHERE Student.sID = Apply.sID and major = 'CS'  -- not numerically correct!

SELECT avg(GPA)
FROM Student, Apply
WHERE sID in (SELECT sID FROM Apply WHERE major = 'CS') -- this is correct!
```

- count

```sql
/*
Number of Colleges with big enrollment
*/
SELECT count(*)
FROM College
WHERE enrollment > 15000

/*
Number of students applie Cornell college
*/
SELECT count(distinct sID)
FROM Apply
WHERE cName = 'Cornell'

/*
Students such that number of other students with same GPA
is equal to number of other students with same sizeHS
*/
SELECT * 
FROM Student S1
WHERE (
    select count(*)
    from Student S2
    where S2.sID <> S1.sID and S2.GPA = S1.GPA
) = (
    select count(*)
    from Student S2
    where S2.sID <> S1.sID and S2.sizeHS = S1.sizeHS
)
```

- arithmetic

```sql
/*
Amount by which average GPA by students applying to CS
exceeds average of students not applying CS
*/
SELECT CS.avgGP - NonCS.avgGPA
FROM (
    SELECT avg(GPA) as avgGPA
    FROM Student
    WHERE sID in (SELECT sID FROM Apply WHERE major = 'CS')
) CS, (
    SELECT avg(GPA) as avgGPA
    FROM Student
    WHERE sID not in (SELECT sID FROM Apply WHERE major = 'CS')
) NonCS;

-- Same as:

SELECT distinct (
    SELECT avg(GPA) as avgGPA
    FROM Student
    WHERE sID in (SELECT sID FROM Apply WHERE major = 'CS')
) - (
    SELECT avg(GPA) as avgGPA
    FROM Student
    WHERE sID not in (SELECT sID FROM Apply WHERE major = 'CS')
)
FROM Studetnt
```

- `group by` and `order by`

```sql
/*
Number of applicants to each colleges
*/
SELECT cName, count(*)
FROM Apply
GROUP BY cName
ORDER BY cName

/*
College enrollments by state
*/
SELECT state, sum(enrollment)
FROM College
GROUP BY state

/*
Minimum and maximum of GPA in each college major, and the difference
*/
SELECT *, (maxGPA - minGPA) as diff
FROM (
    SELECT cName, major, min(GPA) as minGPA, max(GPA) as maxGPA
    FROM Student, Apply
    WHERE Student.sID = Apply.sID
    GROUP BY cName, major
)

/*
Number of colleges applied to by each students
*/
SELECT Student.ID, sName, count(distinct cName)
FROM Apply, Student
WHERE Apply.sID = Student.sID
GROUP BY Student.ID
union
SELECT sID, sName, 0
FROM Student
WHERE sID not in (SELECT sID FROM Apply);
```

- having

```sql
/*
colleges with fewer than 5 applicantions
*/
SELECT cName
FROM Apply
GROUP BY cName
HAVING count(*) < 5

-- Same as:

SELECT distinct cName
FROM Apply A1
WHERE (
    SELECT count(*)
    FROM Apply A2 
    WHERE A2.cName = A1.cName
) < 5

/*
colleges with fewer than 5 applicants
*/
SELECT cName
FROM Apply
GROUP BY cName
HAVING count(distinct sID) < 5

/*
majors whose applicant's maximum GPA is below the average
*/
SELECT major
FROM Student S, Apply A
WHERE S.sID = A.sID
GROUP BY major
HAVING max(GPA) < (
    SELECT avg(GPA)
    FROM Student
)
```

## NULL Values


- undefined or unkown

```sql
SELECT sID, sName, GPA
FROM Student
WHERE GPA <= 3.5 or GPA is null;

/*
Count sdutents with GPAs
*/
SELECT count(*)
FROM Student
WHERE GPA is not null

/*
Count distinct GPAs, won't include null
*/
SELECT count(distinct GPA)  -- won't include null
FROM Student
WHERE GPA is not null

SELECT count(distinct GPA)  -- is the same as above
FROM Student
```

## Data Modification Statements

### Insert

```sql
/*
Insert new college
*/
INSERT INTO College VALUES ('Carnegie Mellon', 'PA', 11600);

/*
instert who has not applied to apply
*/
INSERT INTO Apply
SELECT sID, 'Carnegie Mellon', 'CS', NULL
FROM Student
WHERE sID not in (SELECT sID FROM Apply);

/*
insert who was rejected applying EE to Carnegie EE
*/
INSERT INTO Appy
SELECT sID, 'Carnegie Mellon', 'EE', 'Y'
FROM Student
WHERE sID not in (SELECT sID FROM Apply WHERE major = 'EE' and decision = 'N')
```

### Delete

```sql
/*
Delete all students who applied more than 2 different majors
*/
DELETE FROM Student
WHERE sID in (
    SELECT sID
    FROM Apply
    GROUP BY sID
    HAVING count(distinct major) > 2
)

DELETE FROM Apply
WHERE sID in (
    SELECT sID
    FROM Apply
    GROUP BY sID
    HAVING count(distinct major) > 2
)

/*
Delete colleges with no CS applicants
*/
DELETE FROM College
WHERE cName not in (SELECT cName FROM Apply WHERE major ='CS')
```

### Update

```sql
/*
Accept applicants to Carnegie Mellon with GPA < 3.6
but turn them into EE major
*/
UPDATE Apply
SET decision = 'Y', major = 'EE'
WHERE cName = 'Carnegie Mellon'
    and sID in (SELECT sID FROM Student WHERE GPA < 3.6);
    
/*
Turn the highest-GPA EE applicant into CSE applicant
*/
UPDATE Apply
SET major = 'CSE'
WHERE major = 'EE'
    and sID in (
        SELECT sID FROM Student WHERE GPA >= all(
            SELECT GPA FROM Student WHERE sID in (
                SELECT sID FROM Apply WHERE major = 'EE'
            ) 
        )
    )
    
/*
give every student the highest GPA and the smallest high school in the database.
*/
UPDATE Student
SET GPA = (
    SELECT max(GPA)
    FROM Student
), sizeHS = (
    SELECT min(sizeHS)
    FROM Student
);

/*
accept all the students
*/
UPDATE Student
SET decision = 'Y'
```